In [78]:
import pandas as pd
import os
import json
from concurrent.futures import ThreadPoolExecutor
import requests
import re
import time 
import shutil

In [46]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from utilities.utilities import element_exists 

In [ ]:
chrome_options = ChromeOptions()
chrome_options.add_experimental_option("prefs", {'profile.default_content_setting_values.automatic_downloads': 1})
service = ChromeService(executable_path=ChromeDriverManager().install())

In [48]:
# link where to download data
link = 'https://app.virtuals.io/virtuals/3798/contribution'

In [ ]:

driver = webdriver.Chrome(options=chrome_options, service=service)
driver.get(link)
driver.maximize_window()
time.sleep(5)

In [50]:
# /html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
# /html/body/div[1]/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
# /html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
# /html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
contribs_btn = driver.find_element(By.XPATH, '/html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]')
contribs_btn.click()

In [51]:
element_exists(driver, By.XPATH, '/html/body/path')

False

In [52]:
# select all div elements first
list_container = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[3]/div/div/div/div/div[1]')
divs = list_container.find_elements(By.CSS_SELECTOR, 'div.MuiStack-root')
divs_to_fine_tune = [div.find_element(By.CSS_SELECTOR, 'div:first-child > div:first-child') for div in divs if element_exists(div, By.CSS_SELECTOR, 'div:first-child > div:first-child')]# and ("dataset" in div.find_element(By.CSS_SELECTOR, 'div:first-child > div:first-child').text)]

# check each div elements have its decendant span.MuiChip-label if any 

In [53]:
filtered = list(filter(lambda div: re.search(r"dataset#[0-9]{1,}", div.text.lower()), divs_to_fine_tune))
filtered

[<selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.113")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.113")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.193")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.117")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.117")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053af

In [54]:
for div in filtered:
    text = re.sub(r"[\n]", " ", div.text)
    print(re.search(r"dataset#[0-9]{1,}", div.text.lower()))
    print(f'{text}\n')

<re.Match object; span=(0, 12), match='dataset#6421'>
Dataset#6421 Processing

<re.Match object; span=(0, 12), match='dataset#6421'>
Dataset#6421 Processing

<re.Match object; span=(0, 12), match='dataset#6421'>
Dataset#6421

<re.Match object; span=(0, 12), match='dataset#6420'>
Dataset#6420 Finetune Required

<re.Match object; span=(0, 12), match='dataset#6420'>
Dataset#6420 Finetune Required

<re.Match object; span=(0, 12), match='dataset#6420'>
Dataset#6420

<re.Match object; span=(0, 12), match='dataset#6159'>
Dataset#6159 Finetune Required

<re.Match object; span=(0, 12), match='dataset#6159'>
Dataset#6159 Finetune Required

<re.Match object; span=(0, 12), match='dataset#6159'>
Dataset#6159

<re.Match object; span=(0, 12), match='dataset#5602'>
Dataset#5602 Finetune Required

<re.Match object; span=(0, 12), match='dataset#5602'>
Dataset#5602 Finetune Required

<re.Match object; span=(0, 12), match='dataset#5602'>
Dataset#5602

<re.Match object; span=(0, 12), match='dataset#5484'>


In [55]:
outputs = []
for el in filtered:
    div = el if element_exists(el, By.CSS_SELECTOR, "svg.tabler-icon-download") else None 
    outputs.append(div)
outputs

[<selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.113")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.113")>,
 None,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.117")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.117")>,
 None,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.142")>,
 <selenium.webdriver.remote.webelement.WebElement (session="075244

In [56]:
outputs = list(filter(None, set(outputs)))
outputs

[<selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.113")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.117")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.169")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.142")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0752447fe6719a5053afd12dd965d8e6", element="f.C3377AAFD762561462AD2BBA456FF337.d.BCAABF8A3AD8FF3F8F899AF439426B9B.e.160")>]

In [57]:
for output in outputs:
    print(output.text, end='\n\n')

Dataset#6421
Processing

Dataset#6420
Finetune Required

Dataset#5484
Finetune Required

Dataset#6159
Finetune Required

Dataset#5602
Finetune Required



In [69]:
downloads_path = 'C:/Users/LARRY/Downloads'
filename = max([os.path.join(downloads_path, f) for f in os.listdir(downloads_path)], key=os.path.getctime)
filename

'C:/Users/LARRY/Downloads\\NDA Agreement Larry -PATH SYSTEMS-output.pdf'

In [72]:
base_file_name = filename.split('\\')[-1]
base_file_name

'NDA Agreement Larry -PATH SYSTEMS-output.pdf'

In [73]:
new_file_name = f"dataset#2342{base_file_name}"
new_file_name

'dataset#2342NDA Agreement Larry -PATH SYSTEMS-output.pdf'

In [74]:
os.path.join(downloads_path, new_file_name)

'C:/Users/LARRY/Downloads\\dataset#2342NDA Agreement Larry -PATH SYSTEMS-output.pdf'

In [ ]:
output_dir = './NAME OF DIRECTORY TO STORE INPUT'
os.makedirs(output_dir, exist_ok=True)

dataset_names = []
downloads_path = 'C:/Users/LARRY/Downloads'

for div in outputs:
    text = re.sub(r"[\n]", " ", div.text.lower())
    dataset_num = re.search(r"dataset#[0-9]+", text)[0]
    print(f"downloading {dataset_num}...")

    # extracting clickable svg
    svg = div.find_element(By.CSS_SELECTOR, "svg.tabler-icon-download")
    svg.click()

    # give 2 second delay so file can be downloaded
    time.sleep(3)

    # get most rececntly downloaded file
    file_path = max([os.path.join(downloads_path, f) for f in os.listdir(downloads_path)], key=os.path.getctime)
    base_file_name = file_path.split('\\')[-1]
    new_file_name = f"{dataset_num}{base_file_name}"
    new_file_path = os.path.join(downloads_path, new_file_name)
    os.rename(file_path, new_file_path)

    relocated_path = os.path.join(output_dir, new_file_name)
    shutil.move(new_file_path, relocated_path)

downloading dataset#6421...
downloading dataset#6420...
downloading dataset#5484...
downloading dataset#6159...
downloading dataset#5602...
